In [ ]:
import psycopg2
import random
import faker
import pytz
from datetime import datetime, timedelta
from pytz import timezone
import bcrypt
from psycopg2 import sql, OperationalError, IntegrityError, DataError, InterfaceError

DB_NAME = "DB_Final_Project"
DB_USER = "postgres"
DB_PASSWORD = "bear123321a"
DB_HOST = "localhost" 

# Connect to the PostgreSQL database
# conn = psycopg2.connect(
#     dbname = DB_NAME, 
#     user = DB_USER, 
#     password = DB_PASSWORD, 
#     host = DB_HOST
# )
# cur = conn.cursor()

In [ ]:
def user_create_account(user_id, password, user_name, user_description, profile_pic,
                        profile_background, birthday, email, country, language, fund,
                        filtering, notification, cookies):
    conn = None
    cur = None
    
    try:
        # Establish database connection
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        cur = conn.cursor()

        # Check if the user_id already exists
        cur.execute("""
            SELECT user_id
            FROM public."user"
            WHERE user_id = %s
        """, (user_id,))
        
        # If the user_id already exists, raise an error
        if cur.fetchone():
            print(f"User ID '{user_id}' is already taken.")
            return False
        
        # Hash the password
        encoded_password = password.encode('utf-8')
        hashed_password = bcrypt.hashpw(encoded_password, bcrypt.gensalt())

        # Execute the INSERT query
        query = """
        INSERT INTO public."user"
        (user_id, password_hashed, user_name, user_description, profile_pic,
        profile_background, birthday, email, country, language, fund,
        filtering, notification, cookies)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cur.execute(query, (
            user_id,
            hashed_password,
            user_name,
            user_description,
            profile_pic,
            profile_background,
            birthday,
            email,
            country,
            language,
            fund,
            filtering,
            notification,
            cookies
        ))

        # Commit the transaction
        conn.commit()
        return True

    except OperationalError as e:
        print(f"Database connection error: {e}")
        return False
    except IntegrityError as e:
        print(f"Integrity error (e.g., duplicate user_id): {e}")
        return False
    except DataError as e:
        print(f"Data error (e.g., field value out of range): {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False
    finally:
        # Cleanup resources
        if cur:
            try:
                cur.close()
            except Exception as cleanup_error:
                print(f"Error closing cursor: {cleanup_error}")
        if conn:
            try:
                conn.close()
            except Exception as cleanup_error:
                print(f"Error closing connection: {cleanup_error}")

In [ ]:
def user_login(user_id, password):
    conn = None
    cur = None

    try:
        # Establish database connection
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        cur = conn.cursor()

        # Query for the stored password hash
        cur.execute(
            """
            SELECT password_hashed 
            FROM public."user" 
            WHERE user_id = %s
            """,
            (user_id,)
        )
        stored_password = cur.fetchone()

        # Check if user_id exists
        if not stored_password:
            print("User ID does not exist.")
            return False
        
        # Get the hashed password and validate
        stored_password_hash = stored_password[0]  # Assuming password_hashed is the first column
        if not isinstance(stored_password_hash, (str, bytes)) or len(stored_password_hash) != 60:
            print("Invalid password hash format.")
            return False

        encoded_password = password.encode('utf-8')
        if bcrypt.checkpw(encoded_password, stored_password_hash.encode('utf-8')):
            return True
        else:
            print("Incorrect password.")
            return False

    except OperationalError as e:
        print(f"Database connection error: {e}")
        return False
    except DataError as e:
        print(f"Invalid input data: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False
    finally:
        # Cleanup resources
        if cur:
            try:
                cur.close()
            except Exception as cleanup_error:
                print(f"Error closing cursor: {cleanup_error}")
        if conn:
            try:
                conn.close()
            except Exception as cleanup_error:
                print(f"Error closing connection: {cleanup_error}")

In [ ]:
def show_user_profile(user_id):
    conn = None
    cur = None

    try:
        # Establish database connection
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        cur = conn.cursor()

        # Execute the UPDATE query
        cur.execute(
            """
            SELECT user_id, user_name, user_description, profile_pic, 
                         profile_background, birthday, email, country, 
                         language, fund, filtering, notification, cookies
            FROM public."user"
            WHERE 
                user_id = %s
            """,
            (user_id,)
        )

        # Check if any rows were updated
        if cur.rowcount == 0:
            print("No user found with the specified user_id.")
            return []
        
        user_data = cur.fetchall()

        # Commit the transaction
        conn.commit()
        return user_data

    except OperationalError as e:
        print(f"Database connection error: {e}")
        return []
    except IntegrityError as e:
        print(f"Integrity error (e.g., duplicate or invalid data): {e}")
        return []
    except DataError as e:
        print(f"Invalid input data: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []
    finally:
        # Cleanup resources
        if cur:
            try:
                cur.close()
            except Exception as cleanup_error:
                print(f"Error closing cursor: {cleanup_error}")
        if conn:
            try:
                conn.close()
            except Exception as cleanup_error:
                print(f"Error closing connection: {cleanup_error}")

In [ ]:
def user_add_fund(user_id, amount):
    conn = None
    cur = None

    try:
        # Validate input
        if not isinstance(amount, (int, float)) or amount <= 0:
            print("Invalid amount. Amount must be a positive number.")
            return False

        # Establish database connection
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        cur = conn.cursor()

        # Execute the UPDATE query with row locking (FOR UPDATE)
        cur.execute(
            """
            UPDATE public."user" 
            SET fund = fund + %s
            WHERE user_id = %s
            RETURNING fund;
            """,
            (amount, user_id)
        )

        # Check if the user exists and the update was successful
        updated_row = cur.fetchone()
        if updated_row is None:
            print("User ID not found.")
            return False
        
        cur.execute(
            """
            INSERT INTO public."add_fund_record"
            (user_id, fund_change, timestamp)
            VALUES (%s, %s, NOW())
            """, 
            (user_id, amount)
        )

        # Commit the transaction
        conn.commit()
        print(f"Fund updated successfully. New fund balance: {updated_row[0]}")
        return True

    except OperationalError as e:
        print(f"Database connection error: {e}")
        return False
    except IntegrityError as e:
        print(f"Integrity error (e.g., constraint violation): {e}")
        return False
    except DataError as e:
        print(f"Invalid input data: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False
    finally:
        # Cleanup resources
        if cur:
            try:
                cur.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing cursor: {cleanup_error}")
        if conn:
            try:
                conn.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing connection: {cleanup_error}")

In [ ]:
def user_search_for_games(keywords):
    conn = None
    cur = None

    try:
        # Validate input
        if not isinstance(keywords, str) or not keywords.strip():
            print("Invalid keywords. Keywords must be a non-empty string.")
            return []

        # Establish database connection
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        cur = conn.cursor()

        # Execute the SELECT query
        cur.execute(
            """
            SELECT game_id, game_name
            FROM public."game"
            WHERE game_name ILIKE %s
            """,
            (f"%{keywords}%",)
        )

        # Fetch all matching rows
        related_games = cur.fetchall()

        # Return the results
        return related_games  # Extract game names from tuples

    except OperationalError as e:
        print(f"Database connection error: {e}")
        return []
    except DataError as e:
        print(f"Invalid input data: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []
    finally:
        # Cleanup resources
        if cur:
            try:
                cur.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing cursor: {cleanup_error}")
        if conn:
            try:
                conn.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing connection: {cleanup_error}")

In [ ]:
def user_buy_items(user_id, game_id, item_id):
    conn = None
    cur = None

    try:
        # Validate inputs
        if not all([user_id, game_id, item_id]):
            raise ValueError("Invalid input. Please ensure all inputs are valid.")

        # Establish database connection
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        cur = conn.cursor()

        # Fetch user fund with row locking
        cur.execute(
            """
            SELECT fund 
            FROM public."user" 
            WHERE user_id = %s FOR UPDATE;
            """,
            (user_id,)
        )
        user_result = cur.fetchone()
        if user_result is None:
            raise ValueError("User not found.")
        user_fund = user_result[0]

        # Fetch item price with shared lock
        cur.execute(
            """
            SELECT current_price 
            FROM public."game_item" 
            WHERE game_id = %s AND item_id = %s FOR SHARE;
            """,
            (game_id, item_id)
        )
        item_result = cur.fetchone()
        if item_result is None:
            raise ValueError("Game item not found.")
        current_price = item_result[0]

        # Check if the user has sufficient funds
        if user_fund < current_price:
            raise ValueError("Insufficient funds.")

        # Deduct the item price from user funds
        cur.execute(
            """
            UPDATE public."user" 
            SET fund = fund - %s 
            WHERE user_id = %s;
            """,
            (current_price, user_id)
        )

        # Add transaction
        cur.execute(
            """
            INSERT INTO public."buy_item" 
            (user_id, game_id, item_id, price, timestamp, isCancelled) 
            VALUES (%s, %s, %s, %s, NOW(), %s)
            """,
            (user_id, game_id, item_id, current_price, False)
        )

        # Commit transaction
        conn.commit()
        return True

    except ValueError as e:
        print(f"Validation error: {e}")
        if conn:
            conn.rollback()
        return False
    except OperationalError as e:
        print(f"Database connection error: {e}")
        return False
    except IntegrityError as e:
        print(f"Integrity error (e.g., constraint violation): {e}")
        if conn:
            conn.rollback()
        return False
    except DataError as e:
        print(f"Invalid input data: {e}")
        if conn:
            conn.rollback()
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        if conn:
            conn.rollback()
        return False
    finally:
        # Cleanup resources
        if cur:
            try:
                cur.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing cursor: {cleanup_error}")
        if conn:
            try:
                conn.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing connection: {cleanup_error}")

In [ ]:
def user_uninstall_games(user_id, game_id, uninstalled_date):
    conn = None
    cur = None

    try:
        # Validate inputs
        if not all([user_id, game_id, uninstalled_date]):
            raise ValueError("Invalid input. All parameters (user_id, game_id, uninstalled_date) are required.")

        # Establish database connection
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        cur = conn.cursor()

        # Update uninstalled_date for the game
        cur.execute(
            """
            UPDATE public."user_games"
            SET uninstalled_date = %s
            WHERE user_id = %s AND game_id = %s
            RETURNING user_id, game_id;
            """,
            (uninstalled_date, user_id, game_id)
        )

        # Verify the update was successful
        result = cur.fetchone()
        if result is None:
            raise ValueError("No matching record found for the provided user_id and game_id.")

        # Commit transaction
        conn.commit()
        print(f"Game uninstalled successfully for user_id: {user_id}, game_id: {game_id}")
        return True

    except ValueError as e:
        print(f"Validation error: {e}")
        if conn:
            conn.rollback()
        return False
    except OperationalError as e:
        print(f"Database connection error: {e}")
        return False
    except IntegrityError as e:
        print(f"Integrity error (e.g., constraint violation): {e}")
        if conn:
            conn.rollback()
        return False
    except DataError as e:
        print(f"Invalid input data: {e}")
        if conn:
            conn.rollback()
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        if conn:
            conn.rollback()
        return False
    finally:
        # Cleanup resources
        if cur:
            try:
                cur.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing cursor: {cleanup_error}")
        if conn:
            try:
                conn.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing connection: {cleanup_error}")

In [ ]:
def user_add_friends(user_id, friend_id):
    conn = None
    cur = None

    try:
        # Validate inputs
        if not all([user_id, friend_id]):
            raise ValueError("Both user_id and friend_id are required.")
        if user_id == friend_id:
            raise ValueError("A user cannot add themselves as a friend.")

        # Establish database connection
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        cur = conn.cursor()

        # Add friendship in both directions
        cur.execute(
            """
            INSERT INTO public."user_friends" (user_id, friend_id)
            VALUES (%s, %s)
            ON CONFLICT (user_id, friend_id) DO NOTHING;
            """,
            (user_id, friend_id)
        )
        cur.execute(
            """
            INSERT INTO public."user_friends" (user_id, friend_id)
            VALUES (%s, %s)
            ON CONFLICT (user_id, friend_id) DO NOTHING;
            """,
            (friend_id, user_id)
        )

        # Commit transaction
        conn.commit()
        print(f"Friendship added successfully between user_id: {user_id} and friend_id: {friend_id}.")
        return True

    except ValueError as e:
        print(f"Validation error: {e}")
        if conn:
            conn.rollback()
        return False
    except OperationalError as e:
        print(f"Database connection error: {e}")
        return False
    except IntegrityError as e:
        print(f"Integrity error (e.g., constraint violation): {e}")
        if conn:
            conn.rollback()
        return False
    except DataError as e:
        print(f"Invalid input data: {e}")
        if conn:
            conn.rollback()
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        if conn:
            conn.rollback()
        return False
    finally:
        # Cleanup resources
        if cur:
            try:
                cur.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing cursor: {cleanup_error}")
        if conn:
            try:
                conn.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing connection: {cleanup_error}")

In [ ]:
def user_add_shopping_cart(user_id, game_id, item_id):
    conn = None
    cur = None
    try:
        # Validate inputs
        if not all([user_id, game_id, item_id]):
            raise ValueError("All parameters (user_id, game_id, item_id) are required.")

        # Establish database connection
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        cur = conn.cursor()

        # Insert into shopping cart
        cur.execute(
            """
            INSERT INTO public."cart" (user_id, game_id, item_id)
            VALUES (%s, %s, %s)
            ON CONFLICT (user_id, game_id, item_id) DO NOTHING;
            """,
            (user_id, game_id, item_id)
        )

        # Commit transaction
        conn.commit()
        print(f"Item (game_id: {game_id}, item_id: {item_id}) added to user {user_id}'s shopping cart.")
        return True

    except ValueError as e:
        print(f"Validation error: {e}")
        if conn:
            conn.rollback()
        return False
    except OperationalError as e:
        print(f"Database connection error: {e}")
        return False
    except IntegrityError as e:
        print(f"Integrity error (e.g., constraint violation): {e}")
        if conn:
            conn.rollback()
        return False
    except DataError as e:
        print(f"Invalid input data: {e}")
        if conn:
            conn.rollback()
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        if conn:
            conn.rollback()
        return False
    finally:
        # Cleanup resources
        if cur:
            try:
                cur.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing cursor: {cleanup_error}")
        if conn:
            try:
                conn.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing connection: {cleanup_error}")

In [ ]:
def user_delete_shopping_cart(user_id, game_id, item_id):
    conn = None
    cur = None
    try:
        # Validate inputs
        if not all([user_id, game_id, item_id]):
            raise ValueError("All parameters (user_id, game_id, item_id) are required.")

        # Establish database connection
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        cur = conn.cursor()

        # Execute DELETE statement
        cur.execute(
            """
            DELETE FROM public."cart"
            WHERE user_id = %s AND game_id = %s AND item_id = %s
            """,
            (user_id, game_id, item_id)
        )

        # Commit transaction
        conn.commit()

        # Check if rows were affected
        if cur.rowcount == 0:
            print(f"No matching item found in the shopping cart for user_id: {user_id}, game_id: {game_id}, item_id: {item_id}.")
            return False
        
        print(f"Item (game_id: {game_id}, item_id: {item_id}) successfully deleted from user {user_id}'s shopping cart.")
        return True

    except ValueError as e:
        print(f"Validation error: {e}")
        if conn:
            conn.rollback()
        return False
    except OperationalError as e:
        print(f"Database connection error: {e}")
        return False
    except IntegrityError as e:
        print(f"Integrity error: {e}")
        if conn:
            conn.rollback()
        return False
    except DataError as e:
        print(f"Invalid input data: {e}")
        if conn:
            conn.rollback()
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        if conn:
            conn.rollback()
        return False
    finally:
        # Cleanup resources
        if cur:
            try:
                cur.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing cursor: {cleanup_error}")
        if conn:
            try:
                conn.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing connection: {cleanup_error}")

In [ ]:
def user_review_games(user_id, game_id, rating, text, review_timestamp):
    conn = None
    cur = None
    try:
        # Validate inputs
        if not all([user_id, game_id, rating, text, review_timestamp]):
            raise ValueError("All parameters (review_id, user_id, game_id, rating, text, review_timestamp) are required.")

        # Establish database connection
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        cur = conn.cursor()

        # Execute INSERT statement
        cur.execute(
            """
            INSERT INTO public."game_reviews" 
            (user_id, game_id, rating, text, review_timestamp) 
            VALUES (%s, %s, %s, %s, %s, %s)
            """,
            (user_id, game_id, rating, text, review_timestamp)
        )

        # Commit transaction
        conn.commit()

        # Check if the insert was successful
        if cur.rowcount == 0:
            print("Failed to insert review into the database.")
            return False

        print(f"Review for game_id {game_id} successfully added by user {user_id}.")
        return True

    except ValueError as e:
        print(f"Validation error: {e}")
        if conn:
            conn.rollback()
        return False
    except OperationalError as e:
        print(f"Database connection error: {e}")
        if conn:
            conn.rollback()
        return False
    except IntegrityError as e:
        print(f"Integrity error (e.g., constraint violation): {e}")
        if conn:
            conn.rollback()
        return False
    except DataError as e:
        print(f"Invalid input data: {e}")
        if conn:
            conn.rollback()
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        if conn:
            conn.rollback()
        return False
    finally:
        # Cleanup resources
        if cur:
            try:
                cur.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing cursor: {cleanup_error}")
        if conn:
            try:
                conn.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing connection: {cleanup_error}")

In [ ]:
def user_look_up_for_achievements(game_id):
    conn = None
    cur = None
    try:
        # Validate the input
        if not game_id:
            raise ValueError("Game ID is required.")

        # Establish database connection
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        cur = conn.cursor()

        # Execute the SELECT query to fetch achievements
        cur.execute(
            """
            SELECT achievement_name, achievement_description
            FROM public."game_developers" 
            WHERE game_id = %s;
            """,
            (game_id,)
        )

        # Fetch all results
        achievements = cur.fetchall()

        # If no achievements found, notify the user
        if not achievements:
            print(f"No achievements found for game_id: {game_id}.")
            return None

        # Return the achievements
        return achievements

    except ValueError as e:
        print(f"Validation error: {e}")
        return None
    except OperationalError as e:
        print(f"Database connection error: {e}")
        return None
    except DataError as e:
        print(f"Invalid data error: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        # Cleanup resources
        if cur:
            try:
                cur.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing cursor: {cleanup_error}")
        if conn:
            try:
                conn.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing connection: {cleanup_error}")

In [ ]:
def list_game_items(game_id):
    conn = None
    cur = None

    try:

        # Establish database connection
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        cur = conn.cursor()

        # Execute the SELECT query
        cur.execute(
            """
            SELECT game_id, item_id, current_price
            FROM public."game_item"
            WHERE game_id = %s
            """,
            (game_id,)
        )

        # Fetch all matching rows
        game_items = cur.fetchall()

        # Return the results
        return game_items

    except OperationalError as e:
        print(f"Database connection error: {e}")
        return []
    except DataError as e:
        print(f"Invalid input data: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []
    finally:
        # Cleanup resources
        if cur:
            try:
                cur.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing cursor: {cleanup_error}")
        if conn:
            try:
                conn.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing connection: {cleanup_error}")

In [ ]:
def show_cart(user_id):
    conn = None
    cur = None

    try:

        # Establish database connection
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST
        )
        cur = conn.cursor()

        # Execute the SELECT query
        cur.execute(
            """
            SELECT i.game_id, i.item_id, i.current_price
            FROM public."cart" AS c
            JOIN public."game_item" AS i ON c.game_id = i.game_id AND c.item_id = i.item_id
            WHERE user_id = %s
            """,
            (user_id,)
        )

        # Fetch all matching rows
        game_items_in_cart = cur.fetchall()

        # Return the results
        return game_items_in_cart

    except OperationalError as e:
        print(f"Database connection error: {e}")
        return []
    except DataError as e:
        print(f"Invalid input data: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []
    finally:
        # Cleanup resources
        if cur:
            try:
                cur.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing cursor: {cleanup_error}")
        if conn:
            try:
                conn.close()
            except InterfaceError as cleanup_error:
                print(f"Error closing connection: {cleanup_error}")